## Single Atom Diffusion Scaling Law

Author: Yifan Wang (wangyf@udel.edu)

The scaling law correlates the diffusion activation energy for support single atom on the support with physcial descriptors

The scaling law is obtained from various machine learning methods

The primary physical descriptors (features) include Ec (cohesive energy of the metal) and Ebind (the binding energy of the single atom)

The secondary and 

In [11]:
#%% Import all libraries 

import os
import pickle
import numpy as np
import pandas as pd
from scipy.stats import norm

import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
 

from sklearn import linear_model
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.linear_model import (ElasticNet, ElasticNetCV, Lasso, LassoCV,
                                  Ridge, RidgeCV, enet_path, lasso_path)
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import (LeaveOneOut, RepeatedKFold,
                                     cross_val_score, train_test_split)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

# import customized plotting functions
import regression_tools as rtools

# Set plotting format
font = {'size'   : 20}

matplotlib.rc('font', **font)
matplotlib.rcParams['axes.linewidth'] = 1.5
matplotlib.rcParams['xtick.major.size'] = 12
matplotlib.rcParams['xtick.labelsize'] = 16
matplotlib.rcParams['ytick.labelsize'] = 16
matplotlib.rcParams['xtick.major.width'] = 3
matplotlib.rcParams['ytick.major.size'] = 12
matplotlib.rcParams['ytick.major.width'] = 3
matplotlib.rcParams['legend.fontsize'] = 16
matplotlib.rcParams['figure.dpi'] = 300. # set plotting resolution



In [12]:
#%% Import adsorption data from a csv file
data = pd.read_csv('Ea_data.csv', header = 0)

metal = np.array(data['metal'])
support = np.array(data['support'])
Ec = np.array(data['Ec']) # cohesive energy of the metal
Ebind = np.array(data['Ebind']) #binding energy of the single atom
Ea = np.array(data['Ea']) # Diffusion barrier of the single atom onto the support